In [11]:
from sklearn import metrics

from kmeans.gensim_kmeans import KMeans
from preprocess import load_model
from pprint import pprint
import torch

In [12]:
corpus_t, dictionary_t, tfidf_t, lsi_t, mSimilar_t = load_model("./preprocess/model/")
corpus, dictionary, tfidf, lsi, mSimilar = load_model("./preprocess/model_test/")

2021-04-09 20:26:50,141 : INFO : loaded corpus index from ./preprocess/model/corpus.mm.index
2021-04-09 20:26:50,141 : INFO : initializing cython corpus reader from ./preprocess/model/corpus.mm
2021-04-09 20:26:50,141 : INFO : accepted corpus with 9804 documents, 255478 features, 6928768 non-zero entries
2021-04-09 20:26:50,141 : INFO : loading Dictionary object from ./preprocess/model/model.dic
2021-04-09 20:26:50,282 : INFO : Dictionary lifecycle event {'fname': './preprocess/model/model.dic', 'datetime': '2021-04-09T20:26:50.282720', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}
2021-04-09 20:26:50,282 : INFO : loading TfidfModel object from ./preprocess/model/model.tfidf
2021-04-09 20:26:50,413 : INFO : TfidfModel lifecycle event {'fname': './preprocess/model/model.tfidf', 'datetime': '2021-04-09T20:26:50.413234', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.

In [13]:
X = lsi[tfidf[corpus]]
X_t = lsi_t[tfidf_t[corpus_t]]

In [14]:
import numpy as np

X_list = []
for doc in X:
    if not doc:
        tmp_doc = [0.0 for _ in range(X.obj.num_topics)]
    else:
        tmp_doc = [i[1] for i in doc]
        if len(tmp_doc) < X.obj.num_topics:
            tmp_doc += [0 for _ in range(X.obj.num_topics - len(tmp_doc))]
    X_list.append(tmp_doc)

X_array = np.array(X_list)
X_torch = torch.from_numpy(X_array)

In [15]:
import numpy as np

X_list_t = []
for doc in X_t:
    if not doc:
        tmp_doc = [0.0 for _ in range(X_t.obj.num_topics)]
    else:
        tmp_doc = [i[1] for i in doc]
        if len(tmp_doc) < X_t.obj.num_topics:
            tmp_doc += [0 for _ in range(X_t.obj.num_topics - len(tmp_doc))]
    X_list_t.append(tmp_doc)

X_array_t = np.array(X_list_t)
X_torch_t = torch.from_numpy(X_array_t)

In [16]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [17]:
from file_utils import get_files

f_list = get_files("./corpus/test/")
l = np.array([int(i.split("\\")[-1].split("-")[0][1:]) for i in f_list])

In [10]:
max_acc = 0
max_i = -1
from training_utils import best_map

# train 629
# test 98
# 25,30->1036
for i in range(2000):
    kmeans = KMeans(20, random_state=i, device=device)
    label, _ = kmeans.fit_predict(X, mSimilar, X_array, X_torch)
    acc, _ = best_map(l, label.numpy())
    if acc > max_acc:
        max_acc = acc
        max_i = i
    if acc > 0.48:
        print(i, acc)
print("max", max_i, max_acc)

14 0.4898291293734744
35 0.49705044751830757
158 0.49196501220504474
229 0.4950162733930024
446 0.48423515052888527
535 0.4855573637103336
565 0.48657445077298617
636 0.5058991049633849
642 0.48026851098454026
712 0.48576078112286414
821 0.48871033360455657
857 0.48942229454841335
908 0.48108218063466235
981 0.48891375101708706
986 0.48443856794141577
1033 0.4968470301057771
1036 0.50793327908869
1089 0.4926769731489015
1163 0.48687957689178196
1206 0.4906427990235964
1261 0.4883034987794955
1278 0.4898291293734744
1284 0.48708299430431246
1397 0.49389747762408465
1423 0.4904393816110659
1523 0.4869812855980472
1560 0.48057363710333606
1641 0.4918633034987795
1653 0.48423515052888527
1660 0.4989829129373474
1667 0.4890154597233523
1884 0.48382831570382423
max 1036 0.50793327908869


In [26]:
from training_utils import best_map
from kmeans.gensim_kmeans import KMeans

kmeans = KMeans(20, random_state=98, device=device)
label, _ = kmeans.fit_predict(X, mSimilar, X_array, X_torch)
acc, label_pred = best_map(l, label.numpy())
print(acc)

0.5020341741253052


In [24]:
from training_utils import best_map

kmeans = KMeans(20, random_state=629, device=device)
model = kmeans.fit(X_t, mSimilar_t, X_array_t, X_torch_t)
label_t = model.predict(mSimilar)
acc, l_pred = best_map(l, label_t.numpy())
print(acc)


0.47620016273393


In [27]:
info = metrics.mutual_info_score(l, label_pred)
mutual_info = metrics.adjusted_mutual_info_score(l, label_pred)
normal_info = metrics.normalized_mutual_info_score(l, label_pred)
print("互信息：{0}\n调整互信息：{1}\n标准化互信息：{2}".format(info, mutual_info, normal_info))
rand = metrics.adjusted_rand_score(l, label_pred)
print('兰德系数：', rand)

互信息：1.443762019930632
调整互信息：0.5658882037470625
标准化互信息：0.569235759920096
兰德系数： 0.4412628417309452


In [7]:
import numpy as np
import torch


class KMeans(object):
    def __init__(self,
                 n_clusters,
                 tol=1e-4,
                 max_iter=300,
                 random_state=0,
                 device=torch.device('cpu')):
        self.cluster_centers = []
        self.device = device
        self.n_clusters = n_clusters
        self.tol = tol
        self.max_iter = max_iter
        self.choice_cluster = None
        self.initial_state = None
        self.random_state = random_state
        self.mSimilar = None

    def get_distance(self, initial_state):
        dis = self.mSimilar[initial_state]
        dis = dis.T
        return torch.from_numpy(dis).to(self.device)

    def fit_predict(self, X, mSimilar, X_array=None, X_torch=None):
        self.mSimilar = mSimilar
        if X_array is None:
            X_array = np.array([[i[1] for i in doc] for doc in X])
        if X_torch is None:
            X_torch = torch.from_numpy(X_array).to(self.device)
        num_samples = X.corpus.corpus.num_docs
        np.random.seed(self.random_state)
        indices = np.random.choice(num_samples, self.n_clusters, replace=False)
        dis = self.get_distance(X[indices])
        choice_points = np.array(torch.argmax(dis, dim=0))
        initial_state = X_torch[choice_points]
        choice_points = X[choice_points]
        iteration = 0
        while True:
            dis = self.get_distance(choice_points)
            choice_cluster = torch.argmax(dis, dim=1)
            initial_state_pre = initial_state.clone()
            for index in range(self.n_clusters):
                selected = torch.nonzero(torch.from_numpy(np.array(choice_cluster == index))).squeeze().to(self.device)
                selected = torch.index_select(X_torch, 0, selected)
                if selected.shape[0] == 0:
                    selected = X_torch[torch.randint(len(X), (1,))]

                initial_state[index] = selected.mean(dim=0)
            center_shift = torch.sum(
                torch.sqrt(
                    torch.sum((initial_state - initial_state_pre) ** 2, dim=1)
                ))
            choice_points = [[(j, np.array(initial_state[i]).tolist()[j]) for j in range(20)] for i in
                             range(len(initial_state))]
            iteration = iteration + 1

            if center_shift ** 2 < self.tol:
                break
            if self.max_iter != 0 and iteration >= self.max_iter:
                break
            self.initial_state = initial_state.cpu()
        return choice_cluster.cpu(), initial_state.cpu()

    def fit(self, X, mSimilar, X_array=None, X_torch=None):
        self.mSimilar = mSimilar
        num_samples = X.corpus.corpus.num_docs
        if X_array is None:
            X_array = np.array([[i[1] for i in doc] for doc in X])
        if X_torch is None:
            X_torch = torch.from_numpy(X_array).to(self.device)
        np.random.seed(self.random_state)
        indices = np.random.choice(num_samples, self.n_clusters, replace=False)
        dis = self.get_distance(X[indices])
        choice_points = np.array(torch.argmax(dis, dim=0))
        self.initial_state = X_torch[choice_points]
        choice_points = X[choice_points]
        iteration = 0
        while True:
            dis = self.get_distance(choice_points)
            choice_cluster = torch.argmax(dis, dim=1)
            initial_state_pre = self.initial_state.clone()
            for index in range(self.n_clusters):
                selected = torch.nonzero(
                    torch.from_numpy(np.array(choice_cluster == index)).to(self.device)).squeeze().to(self.device)
                selected = torch.index_select(X_torch, 0, selected)
                if selected.shape[0] == 0:
                    selected = X_torch[torch.randint(len(X), (1,))]
                self.initial_state[index] = selected.mean(dim=0)
            center_shift = torch.sum(
                torch.sqrt(
                    torch.sum((self.initial_state - initial_state_pre) ** 2, dim=1)
                ))
            choice_points = [[(j, np.array(self.initial_state[i]).tolist()[j]) for j in range(20)] for i in
                             range(len(self.initial_state))]
            iteration = iteration + 1

            if center_shift ** 2 < self.tol:
                break
            if self.max_iter != 0 and iteration >= self.max_iter:
                break

        self.cluster_centers = self.initial_state
        return self

    def predict(self, mSimilar):
        tmp = self.mSimilar
        self.mSimilar = mSimilar
        choice_points = [[(j, np.array(self.initial_state[i]).tolist()[j]) for j in range(20)] for i in
                         range(len(self.initial_state))]
        dis = self.get_distance(choice_points)
        choice_cluster = torch.argmax(dis, dim=1)
        self.mSimilar = tmp
        return choice_cluster.cpu()

In [29]:
max_acc = 0
max_i = -1
from training_utils import best_map

# train 629
# test 98
# train test 102 202

kmeans = KMeans(20, random_state=202, device=device)
model = kmeans.fit(X_t, mSimilar_t, X_array_t, X_torch_t)
label_t = model.predict(mSimilar)
acc, _ = best_map(l, label_t.numpy())

print(acc)

0.5213588283157038
